### Config stuff

In [1]:
import os

import ConnectionConfigKaloyan as cc
from delta import DeltaTable
cc.setupEnvironment()

In [2]:
spark = cc.startLocalCluster("FACT_RIDE")
spark.getActiveSession()

# Fact transformations
This notebooks creates the sales fact table from scratch based on the operational source table "sales"
When creating a fact table always follow the listed steps in order.


#### 1 READ NECESSARY SOURCE TABLE(S) AND PERFORM TRANSFORMATIONS
**When reading from the source table make sure you include all data necessary:**
- to calculate the measure values
- the source table keys that you have to use to lookup the correct surrogate keys in the dimension tables.

**If more than one table is needed to gather the necesary information you can opt for one of two strategies:**
- Use a select query when reading from the jdbc source with the spark.read operation. Avoid complex queries because the operational database needs a lot of resources to run those queries.
- Perform a spark.read operation for each table separately and join the tables within Spark. The joins will take place on the cluster instead of the database. You limit the database recources used, but there can be a significant overhead of unnecessary data tranferred to the cluster.


In this case we just rename Amount and create a default count_mv column.
The transformations are minimal. In reality, transformations can be far more complex. If so, it can be advisable to work out the transforms in more then one step.*



In [9]:
cc.set_connectionProfile("default")
ride_src_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","rides").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()

subscriptions_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","subscriptions").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "subscriptionid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()

weather_df = spark.read.json(r'C:\Users\kkiva\data4_project_group5\examples\weather\*.json')


In [13]:
weather_df.show(20)

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().


#### 2 MAKE DIMENSION TABLES AVAILABLE AS VIEWS

In [4]:
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")
dim_vehicle = spark.read.format("delta").load("spark-warehouse/dimvehicle")
dim_user = spark.read.format("delta").load("spark-warehouse/dimuser")
dim_weather = spark.read.format("delta").load("spark-warehouse/dimuser")
dim_lock = spark.read.format("delta").load("spark-warehouse/dimlock")

dim_date.createOrReplaceTempView("dimDate")
dim_user.createOrReplaceTempView("dimUser")
dim_vehicle.createOrReplaceTempView("dimVehicle")
dim_weather.createOrReplaceTempView("dimWeather")
dim_lock.createOrReplaceTempView("dimLock")

Py4JJavaError: An error occurred while calling o82.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (DESKTOP-DJQSI3S executor driver): org.apache.spark.SparkException: Encountered error while reading file file:///C:/Users/kkiva/data4_project_group5/examples/spark-warehouse/dimdate/_delta_log/00000000000000000001.json. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:864)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIte


#### 3 Build the fact table

Within the creation of a fact table always perform these two tasks:
1.   Include the measures of the fact
2. Use the dimension tables to look up the surrogate keys that correspond with the natural key value. In case of SCD2 dimension use the scd_start en scd_end to find the correct version of the data in the dimension


In [5]:
ride_src_df.createOrReplaceTempView("rides_source")
ridesFactFromSource = spark.sql("select src.rideid as ride_id, du.userSK as user_sk, src.startlockid as start_lock_ic, src.endlockid as end_lock_id, dd.date_sk as date_sk,dv.vehicleid as vehicle_id,(src.endtime - src.startime) as ride_duration, md5(concat(src.rideid, du.userSK, src.startlockid, src.endlockid, dd.date_sk, dv.vehicleid)) as md5 \
          from rides_source as src \
          left outer join subscriptions_df as sub on src.subscriptionid = sub.subscriptionid \
          left outer join dim_user as du on sub.userid = du.userSK \
          left outer join dim_lock as dl on src.startlockid = dl.lockid and src.endlockid = dl.lockid  \
          left outer join dim_date as dd on DATE(src.starttime) = dd.CalendarDate \
          left outer join dim_vehicle as dv on src.vehicleid = dv.vehicleid \
")

ridesFactFromSource.show(10)
ridesFactFromSource.createOrReplaceTempView("factRides_new")

+-------+------+--------------------+--------+----------+--------------------+
|OrderID|dateSK|          salesrepSK|count_mv|revenue_mv|                 md5|
+-------+------+--------------------+--------+----------+--------------------+
|      1|   650|b65df3d9-20dc-42d...|       1| 851804379|a237b06f2932af7dd...|
|      2|  1369|b65df3d9-20dc-42d...|       1| 683057055|f2f29b4c68d66f8a6...|
|      3|   920|b65df3d9-20dc-42d...|       1|1732115679|ce49bc85950b1de43...|
|      4|   604|b65df3d9-20dc-42d...|       1|1275042249|908be3f822ea3717d...|
|      5|   897|b65df3d9-20dc-42d...|       1| 694153767|fddeb9cfa84ec3a43...|
|      6|   812|b65df3d9-20dc-42d...|       1|1959464599|4c7a00e21fb76c7d8...|
|      7|   421|b65df3d9-20dc-42d...|       1|1170677605|31eb59a94c0a895e2...|
|      8|   691|b65df3d9-20dc-42d...|       1|1588502393|a30a88ea5f72f4df0...|
|      9|  1254|b65df3d9-20dc-42d...|       1|1173163372|01aeb05aafe965913...|
|     10|  1311|b65df3d9-20dc-42d...|       1| 78868

## Initial load
The first time loading the fact table perform a FULL load. All data is written to the Delta Table.
After initial load the code line has to be disabled

In [6]:

ridesFactFromSource.write.format("delta").mode("overwrite").saveAsTable("factRides")



## Incremental load
When previous runs where performend you can opt for a 'faster' incremental run that only writes away changes. UPDATES and INSERTS are performed in one run.
In our solution we use an md5 based on all fields in the source table to detect changes. This is not the most efficient way to detect changes. A better way is to use a timestamp field in the source table and use that to detect changes. This is not implemented in this example.

In [7]:
dt_factRides = DeltaTable.forPath(spark,".\spark-warehouse\\factrides")
dt_factRides.toDF().createOrReplaceTempView("factRides_current")
#Merge to perform updates (TODO: Implement md5 strategy)

result = spark.sql("MERGE INTO factRides_current AS target \
      using factRides_new AS source ON target.rideID = source.rideID \
      WHEN MATCHED and source.MD5<>target.MD5 THEN UPDATE SET * \
      WHEN NOT MATCHED THEN INSERT *")

result.show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                0|               0|               0|                0|
+-----------------+----------------+----------------+-----------------+


In [8]:
# IMPORTANT: ALWAYS TEST THE CREATED CODE.
# In this example I changed order 498 in the operational database and checked the change after the run.
# spark.sql("select * from factsales f join dimsalesrep ds on f.salesrepSK = ds.salesrepSK where OrderID = 192  ").show()
spark.sql("select count(*) from factrides").show()
spark.sql("select * from factrides where rideId=1").show()



+--------+
|count(1)|
+--------+
|     999|
+--------+

+-------+------+--------------------+--------+----------+--------------------+
|OrderID|dateSK|          salesrepSK|count_mv|revenue_mv|                 md5|
+-------+------+--------------------+--------+----------+--------------------+
|      1|   650|b65df3d9-20dc-42d...|       1| 851804379|a237b06f2932af7dd...|
+-------+------+--------------------+--------+----------+--------------------+


### Checking the history of your delta fact table

In [9]:
# The history information is derived from the delta table log files. They contain a lot of information of all the actions performed on the table. In this case it tells us something about de merge operations. You can find statistics about the update and insert counts in the document.

fact.history().show(10,False)

NameError: name 'fact' is not defined

In [13]:
spark.stop()